In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/adv/song_tag_emotion.csv')
df

,song id,tag,emotion
0,0,"비오는날, 드라이브, 비오는날, 추억, 회상","우울한, 행복한, 우울한, 행복한, 잔잔한"
1,3,"드라이브, 여행, 기분전환, 신나는, 클럽, 드라이브, 페스티벌, 기분전환, 스트레...","행복한, 행복한, 행복한, 행복한, 경쾌한, 행복한, 경쾌한, 행복한, 긴장되는, ..."
2,5,"비오는날, 휴식, 힐링, 기분전환","우울한, 잔잔한, 행복한, 행복한"
3,6,"산책, 조깅","행복한, 경쾌한"
4,9,위로,우울한
...,...,...,...
395360,707979,"기분전환, 드라이브","행복한, 행복한"
395361,707980,"겨울, 밤, 카페, 겨울밤, 센치, 휴식, 힐링, 산책","잔잔한, 무서운, 잔잔한, 잔잔한, 우울한, 잔잔한, 행복한, 행복한"
395362,707982,신나는,행복한
395363,707986,"카페, 잔잔한, 추억, 위로, 힐링, 사랑, 이별, 슬픔, 이별, 설렘, 사랑, 잔...","잔잔한, 잔잔한, 행복한, 우울한, 행복한, 행복한, 우울한, 우울한, 우울한, 행..."


## ratring matrix 기본 형태

In [5]:
# 기존 df를 transpose한 형태로 새로운 rating matrix 생성
rating = df.T
# 감정 기준으로 새로 matrix를 만들기 때문에 삭제
rating.drop('tag', inplace = True)
# column을 첫번째 행인 song id로 설정
rating.rename(columns = rating.iloc[0], inplace = True)
# 첫번째 행 삭제
rating.drop('song id', inplace = True)

In [6]:
rating

,0,3,5,6,9,10,11,12,14,16,17,18,19,22,23,28,30,32,34,35,36,39,40,41,47,48,51,54,56,57,58,59,60,62,64,67,68,69,70,71,...,707918,707919,707922,707923,707924,707925,707926,707927,707928,707932,707933,707934,707938,707940,707941,707944,707948,707949,707950,707951,707952,707953,707955,707956,707957,707960,707964,707965,707966,707967,707969,707970,707974,707975,707978,707979,707980,707982,707986,707987
emotion,"우울한, 행복한, 우울한, 행복한, 잔잔한","행복한, 행복한, 행복한, 행복한, 경쾌한, 행복한, 경쾌한, 행복한, 긴장되는, ...","우울한, 잔잔한, 행복한, 행복한","행복한, 경쾌한",우울한,"행복한, 무서운, 편안한, 잔잔한, 우울한, 경쾌한, 잔잔한, 편안한, 잔잔한, 무...","행복한, 잔잔한, 행복한, 행복한, 행복한","잔잔한, 행복한",경쾌한,"긴장되는, 행복한, 편안한, 행복한","경쾌한, 경쾌한, 행복한, 경쾌한, 경쾌한, 행복한, 행복한, 행복한, 잔잔한, 잔...",긴장되는,"우울한, 행복한, 잔잔한, 잔잔한, 잔잔한, 우울한, 우울한, 잔잔한, 행복한, 잔...",행복한,"행복한, 잔잔한, 행복한, 행복한","경쾌한, 행복한, 행복한, 행복한, 경쾌한, 행복한, 행복한, 잔잔한, 행복한, 잔...","행복한, 행복한, 행복한, 행복한, 편안한, 행복한, 행복한, 경쾌한, 경쾌한, 경...","잔잔한, 잔잔한, 잔잔한, 잔잔한, 우울한, 우울한, 행복한, 잔잔한, 행복한, 경...",긴장되는,"편안한, 잔잔한, 행복한, 잔잔한, 잔잔한",행복한,"잔잔한, 행복한, 행복한, 잔잔한, 무서운","행복한, 잔잔한, 긴장되는","잔잔한, 잔잔한, 행복한, 잔잔한, 잔잔한","우울한, 행복한, 잔잔한, 행복한, 우울한, 우울한, 행복한, 잔잔한, 무서운, 잔...","긴장되는, 경쾌한, 행복한, 우울한, 행복한, 잔잔한, 경쾌한, 잔잔한, 잔잔한","잔잔한, 잔잔한, 긴장되는, 잔잔한, 잔잔한, 잔잔한, 잔잔한, 잔잔한, 잔잔한","경쾌한, 잔잔한, 행복한, 행복한, 행복한, 행복한, 행복한, 행복한, 잔잔한, 잔...","무서운, 우울한","행복한, 잔잔한, 무서운, 경쾌한, 행복한","잔잔한, 행복한, 잔잔한","잔잔한, 잔잔한, 행복한, 행복한, 행복한, 행복한, 경쾌한","행복한, 행복한, 잔잔한","잔잔한, 무서운, 우울한, 우울한, 잔잔한, 행복한, 잔잔한, 행복한, 무서운, 우...","행복한, 행복한, 긴장되는, 행복한","행복한, 잔잔한, 긴장되는, 우울한, 우울한, 무서운, 우울한, 잔잔한, 우울한, ...","울고싶은, 잔잔한, 우울한, 우울한, 우울한, 잔잔한, 편안한, 잔잔한, 편안한, ...","행복한, 행복한","우울한, 행복한, 무서운, 잔잔한, 편안한, 잔잔한, 행복한, 우울한, 행복한, 무...","행복한, 잔잔한, 행복한, 행복한, 행복한, 행복한, 잔잔한, 행복한, 잔잔한, 잔...",...,경쾌한,경쾌한,"경쾌한, 행복한, 행복한, 행복한, 잔잔한, 행복한, 행복한, 우울한, 잔잔한, 행...","잔잔한, 잔잔한","경쾌한, 긴장되는, 행복한, 경쾌한, 행복한, 행복한, 행복한, 긴장되는, 행복한","잔잔한, 행복한, 행복한, 우울한, 우울한, 잔잔한, 행복한, 행복한, 잔잔한, 무...","잔잔한, 잔잔한, 행복한, 행복한, 긴장되는, 행복한, 행복한, 긴장되는","잔잔한, 행복한, 잔잔한, 잔잔한, 행복한, 행복한, 잔잔한, 행복한, 무서운, 우...","행복한, 잔잔한","행복한, 행복한","행복한, 잔잔한, 무서운, 우울한","행복한, 행복한, 잔잔한, 행복한, 잔잔한, 행복한, 행복한, 행복한, 잔잔한, 잔...",긴장되는,"경쾌한, 행복한, 잔잔한","잔잔한, 행복한, 행복한, 행복한","잔잔한, 행복한, 행복한, 행복한","경쾌한, 행복한, 우울한, 행복한, 우울한, 행복한","행복한, 행복한, 행복한, 행복한, 행복한, 행복한, 긴장되는, 행복한, 행복한, ...","행복한, 경쾌한, 행복한, 잔잔한, 행복한","경쾌한, 행복한, 경쾌한, 행복한, 행복한","행복한, 잔잔한, 잔잔한, 잔잔한, 무서운, 우울한, 우울한, 무서운, 우울한","긴장되는, 경쾌한, 경쾌한","행복한, 행복한","행복한, 잔잔한, 잔잔한, 행복한, 잔잔한, 긴장되는, 행복한, 행복한, 행복한, 잔잔한",행복한,"행복한, 우울한, 행복한, 행복한, 우울한, 우울한, 편안한, 우울한, 우울한, 우울한","경쾌한, 행복한, 경쾌한, 행복한, 행복한, 행복한","경쾌한, 행복한, 경쾌한, 경쾌한, 경쾌한, 행복한, 행복한, 경쾌한, 잔잔한, 잔...",경쾌한,"행복한, 잔잔한, 잔잔한","행복한, 행복한, 잔잔한, 행복한, 잔잔한, 경쾌한, 잔잔한, 행복한, 행복한, 행...","무서운, 행복한, 행복한, 우울한, 잔잔한, 잔잔한, 잔잔한, 잔잔한, 행복한, 행...","잔잔한, 행복한, 편안한","행복한, 잔잔한, 행복한","행복한, 잔잔한, 우울한, 행복한, 잔잔한, 잔잔한, 행복한, 행복한, 우울한, 잔...","행복한, 행복한","잔잔한, 무서운, 잔잔한, 잔잔한, 우울한, 잔잔한, 행복한, 행복한",행복한,"잔잔한, 잔잔한, 행복한, 우울한, 행복한, 행복한, 우울한, 우울한, 우울한, 행...","잔잔한, 긴장되는, 우울한, 행복한, 잔잔한, 잔잔한"


## 감정 비율 계산

In [8]:
from collections import Counter

def softmax(t):
    s = set(t)
    c = [dict(Counter(t))[name] for name in s]
    return dict(zip(s, np.array(c) / sum(c)))

softmax(rating[0]['emotion'].split(', '))

{'우울한': 0.4, '잔잔한': 0.2, '행복한': 0.4}

In [10]:
idx = ['우울한', '울고싶은', '긴장되는', '무서운', '잔잔한', '행복한', '경쾌한', '편안한']
cols = list(rating.columns) 

In [13]:
all = []

for c in cols:
  s = softmax(rating[c]['emotion'].split(', '))
  emotions = []

  for i in idx:
      try:
          emotions.append(s[i])
      except:
          emotions.append(0)

  # 감정 점수 다 모아서 한번에 column별로 넣을 예정
  all.append(emotions)

## 노래-감정 rating matrix

In [20]:
emotion = pd.DataFrame(all).T
emotion.index = idx
emotion.columns = rating.columns
emotion

,0,3,5,6,9,10,11,12,14,16,17,18,19,22,23,28,30,32,34,35,36,39,40,41,47,48,51,54,56,57,58,59,60,62,64,67,68,69,70,71,...,707918,707919,707922,707923,707924,707925,707926,707927,707928,707932,707933,707934,707938,707940,707941,707944,707948,707949,707950,707951,707952,707953,707955,707956,707957,707960,707964,707965,707966,707967,707969,707970,707974,707975,707978,707979,707980,707982,707986,707987
우울한,0.4,0.00,0.25,0.0,1.0,0.156250,0.0,0.0,0.0,0.00,0.000000,0.0,0.311828,0.0,0.00,0.00,0.020134,0.367347,0.0,0.0,0.0,0.0,0.000000,0.0,0.131579,0.111111,0.000000,0.119512,0.5,0.0,0.000000,0.000000,0.000000,0.327309,0.00,0.37500,0.172414,0.0,0.142857,0.079805,...,0.0,0.0,0.133333,0.0,0.000000,0.272727,0.00,0.142857,0.0,0.0,0.25,0.000000,0.0,0.000000,0.00,0.00,0.333333,0.041667,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.6,0.000000,0.171429,0.0,0.000000,0.162162,0.117647,0.000000,0.000000,0.10,0.0,0.125,0.0,0.235294,0.166667
울고싶은,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.002439,0.0,0.0,0.000000,0.000000,0.000000,0.006024,0.00,0.00000,0.034483,0.0,0.000000,0.001629,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.011905,0.0,0.0,0.00,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.0,0.000000,0.000000
긴장되는,0.0,0.05,0.00,0.0,0.0,0.041667,0.0,0.0,0.0,0.25,0.035714,1.0,0.000000,0.0,0.00,0.05,0.020134,0.061224,1.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.111111,0.111111,0.053659,0.0,0.0,0.000000,0.000000,0.000000,0.014056,0.25,0.03125,0.103448,0.0,0.000000,0.026059,...,0.0,0.0,0.066667,0.0,0.222222,0.000000,0.25,0.023810,0.0,0.0,0.00,0.055556,1.0,0.000000,0.00,0.00,0.000000,0.041667,0.0,0.0,0.000000,0.333333,0.0,0.1,0.0,0.0,0.000000,0.057143,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.0,0.000,0.0,0.000000,0.166667
무서운,0.0,0.00,0.00,0.0,0.0,0.072917,0.0,0.0,0.0,0.00,0.000000,0.0,0.053763,0.0,0.00,0.00,0.006711,0.040816,0.0,0.0,0.0,0.2,0.000000,0.0,0.052632,0.000000,0.000000,0.031707,0.5,0.2,0.000000,0.000000,0.000000,0.092369,0.00,0.07500,0.034483,0.0,0.142857,0.034202,...,0.0,0.0,0.000000,0.0,0.000000,0.090909,0.00,0.071429,0.0,0.0,0.25,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.222222,0.000000,0.0,0.0,0.0,0.0,0.000000,0.028571,0.0,0.000000,0.054054,0.058824,0.000000,0.000000,0.00,0.0,0.125,0.0,0.029412,0.000000
잔잔한,0.2,0.00,0.25,0.0,0.0,0.447917,0.2,0.5,0.0,0.00,0.071429,0.0,0.397849,0.0,0.25,0.20,0.080537,0.265306,0.0,0.6,0.0,0.4,0.333333,0.8,0.289474,0.333333,0.888889,0.329268,0.0,0.2,0.666667,0.285714,0.333333,0.269076,0.00,0.19375,0.275862,0.0,0.357143,0.333876,...,0.0,0.0,0.133333,1.0,0.000000,0.272727,0.25,0.428571,0.5,0.0,0.25,0.222222,0.0,0.333333,0.25,0.25,0.000000,0.041667,0.2,0.0,0.333333,0.000000,0.0,0.4,0.0,0.0,0.000000,0.371429,0.0,0.666667,0.351351,0.411765,0.333333,0.333333,0.35,0.0,0.500,0.0,0.323529,0.500000
행복한,0.4,0.60,0.50,0.5,0.0,0.187500,0.8,0.5,0.0,0.50,0.607143,0.0,0.193548,1.0,0.75,0.60,0.536913,0.204082,0.0,0.2,1.0,0.4,0.333333,0.2,0.342105,0.222222,0.000000,0.426829,0.0,0.4,0.333333,0.571429,0.666667,0.267068,0.75,0.29375,0.206897,1.0,0.285714,0.467427,...,0.0,0.0,0.600000,0.0,0.555556,0.363636,0.50,0.250000,0.5,1.0,0.25,0.611111,0.0,0.333333,0.75,0.75,0.500000,0.833333,0.6,0.6,0.111111,0.000000,1.0,0.5,1.0,0.3,0.666667,0.200000,0.0,0.333333,0.405405,0.411765,0.333333,0.666667,0.45,1.0,0.250,1.0,0.411765,0.166667
경쾌한,0.0,0.35,0.00,0.5,0.0,0.041667,0.0,0.0,1.0,0.00,0.285714,0.0,0.010753,0.0,0.00,0.10,0.322148,0.040816,0.0,0.0,0.0,0.0,0.000000,0.0,0.105263,0.222222,0.000000,0.026829,0.0,0.2,0.000000,0.142857,0.000000,0.014056,0.00,0.02500,0.068966,0.0,0.000000,0.045603,...,1.0,1.0,0.066667,0.0,0.222222,0.000000,0.00,0.011905,0.0,0.0,0.00,0.111111,0.0,0.333333,0.00,0.00,0.166667,0.041667,0.2,0.4,0.000000,0.666667,0.0,0.0,0.0,0.0,0.333333,0.171429,1.0,0.000

In [28]:
emotion.to_csv('/content/drive/MyDrive/Colab Notebooks/adv/emotion rating matrix.csv')